In [0]:
%pip install pytest

In [0]:
import pytest
from pyspark.sql.utils import AnalysisException
from pyspark.sql.functions import col

table_name = "nintendodatabricks_workspace.bronze.consoles_ing"
expected_schema = ["desconto", "codigo", "nome", "preco", "parcelamento", "link", "origem", "extract"]

def check_table_exists(table):
    try:
        spark.table(table)
        return True
    except AnalysisException:
        return False

def check_schema(table, expected_cols):
    df = spark.table(table)
    actual_cols = df.columns
    return set(actual_cols) == set(expected_cols)

def check_nulls(table):
    df = spark.table(table)
    null_counts = {}
    for c in df.columns:
        cnt = df.filter(col(c).isNull()).count()
        null_counts[c] = cnt
    return null_counts

def check_not_empty(table):
    df = spark.table(table)
    return not df.rdd.isEmpty()

@pytest.mark.parametrize("table", [table_name])
def test_table_exists(table):
    assert check_table_exists(table), f"Table {table} should exist"

@pytest.mark.parametrize("table,expected_cols", [(table_name, expected_schema)])
def test_schema(table, expected_cols):
    assert check_schema(table, expected_cols), f"Schema of table {table} does not match expected"

@pytest.mark.parametrize("table", [table_name])
def test_no_nulls(table):
    nulls = check_nulls(table)
    null_columns = [col for col, cnt in nulls.items() if cnt > 0]
    assert not null_columns, f"Columns with null values: {null_columns}"

@pytest.mark.parametrize("table", [table_name])
def test_not_empty(table):
    assert check_not_empty(table), f"Table {table} is empty"

In [0]:
print("Table exists:", check_table_exists(table_name))
print("Schema ok:", check_schema(table_name, expected_schema))
print("Null counts:", check_nulls(table_name))
print("Not empty:", check_not_empty(table_name))


In [0]:
try:
    test_table_exists(table_name)
    test_schema(table_name, expected_schema)
    test_no_nulls(table_name)
    test_not_empty(table_name)
    print("✅ Todos os testes passaram")
except AssertionError as e:
    print(f"❌ Falhou: {e}")
